In [ ]:
!pip install neo4j openai==0.28.1 langchain tiktoken

##Vector similarity recommendation

In [ ]:
from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
  "bolt://3.82.104.182:7687",
  auth=basic_auth("neo4j", "wait-inventions-washers"))

current_title = "Performance Testing Neo4j Database using Bolt Protocol in Apache JMeter"
current_uri = "https://dev.to/qainsights/performance-testing-neo4j-database-using-bolt-protocol-in-apache-jmeter-1oa9"

vector_similarity = '''
match (a:Article { uri: $uri })
call db.index.vector.queryNodes('article-embeddings', 5, a.embedding)
yield node AS similarArticle, score
where  similarArticle <> a
return similarArticle.title as title, similarArticle.body as body, score
'''

with driver.session(database="neo4j") as session:
  results = session.read_transaction(
    lambda tx: tx.run(vector_similarity,
                      uri=current_uri).data())

  context = "Related documents: \n\n" + "\n\n".join(["title: " + record['title'] + "\n" + "body: " + record['body'] + "\n" for record in results ])

driver.close()

print(context)

<ipython-input-61-22911a319dca>:19: DeprecationWarning: read_transaction has been renamed to execute_read
  results = session.read_transaction(


Related documents: 

title: Building a Modern Web Application with Neo4j and NestJS
body: This article is the introduction to a series of articles and a Twitch stream on the Neo4j Twitch channel where I build an application on top of Neo4j with NestJS and a yet-to-be-decided Front End. This week I built a Module and Service for interacting with Neo4j.TL;DR: I've pushed the code to Github and created a Neo4j module for NestJS to save you some time.Over the past few weeks I have been spending an hour live streaming something that I have found interesting that week, but from this week I thought I would change things up and start to build out a project on Neo4j.If you're subscribed to this channel then you are likely familiar with Neo4j, but if not then Neo4j is the world's leading Graph Database. Rather than tables or documents, Neo4j stores it's data in Nodes - those nodes are categorised by labels and contain properties as key/value pairs. Those Nodes are connected together by relations

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0)

template = """I've read an article titled {current_article_title}.
Tell me what should I read next and a brief description of why based only on the provided context.

{context}
"""
prompt = PromptTemplate.from_template(template)

theprompt = prompt.format_prompt(current_article_title=current_title,context=context)

#print(theprompt)

llm(theprompt.to_messages())

content='Based on the provided context, the next document you should read is "Building a Modern Web Application with Neo4j and NestJS." This article is related to the previous article about performance testing Neo4j database using Bolt Protocol in Apache JMeter. It introduces a series of articles and a Twitch stream where the author builds an application on top of Neo4j with NestJS. It discusses the benefits of using Neo4j as a graph database and explains how to use NestJS, a framework for building server-side applications, with Neo4j. This article will provide further insights into using Neo4j in a web application context.'


##Graph similarity recommendation

In [ ]:
from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
  "bolt://3.82.104.182:7687",
  auth=basic_auth("neo4j", "wait-inventions-washers"))

current_title = "Performance Testing Neo4j Database using Bolt Protocol in Apache JMeter"
current_uri = "https://dev.to/qainsights/performance-testing-neo4j-database-using-bolt-protocol-in-apache-jmeter-1oa9"

graph_similarity = '''
match (a:Article { uri: $uri})-[rt1:refers_to]->(c1)
call n10s.sim.pathsim.search(c1,0.2,{ simulateRoot: false }) yield node as relatedTopic
with a,c1,collect(relatedTopic) + [c1] as topics
unwind topics as c2
match (similarArticle:Article )-[rt2:refers_to]->(c2) //where similarArticle <> a
with a.title as original, similarArticle.title as similar,
[x in collect(n10s.sim.pathsim.value(c1,c2, { simulateRoot: false})) where x > 0 ] as sims ,
collect(n10s.sim.pathsim.path(c1,c2, { simulateRoot: false})) as paths,
collect(["the original article mentions explicitly " + nodes(n10s.sim.pathsim.path(c1,c2, { simulateRoot: false}))[0].prefLabel ]  +
 [" the recommended article mentions explicitly " + nodes(n10s.sim.pathsim.path(c1,c2, { simulateRoot: false}))[-1].prefLabel]  + [r in relationships(n10s.sim.pathsim.path(c1,c2, { simulateRoot: false})) | startNode(r).prefLabel + " is a type of " + endNode(r).prefLabel]) as paths_as_text
where sims <> []
return similar as title, apoc.coll.avg(sims) as sim, sims, reduce(result="", x in paths_as_text | result + reduce(inner="", y in x | inner + "\n"+y))  as explanation
order by sim desc limit 4
'''

with driver.session(database="neo4j") as session:
  results = session.read_transaction(
    lambda tx: tx.run(graph_similarity,
                      uri=current_uri).data())

  context = "Related documents: \n\n" + "\n\n".join(["title: " + record['title'] + "\n" + "explanation: " + record['explanation'] + "\n" for record in results ])

driver.close()

print(context)

<ipython-input-69-4d3c7b685ab3>:27: DeprecationWarning: read_transaction has been renamed to execute_read
  results = session.read_transaction(


Related documents: 

title: Building a Modern Web Application with Neo4j and NestJS
explanation: 
the original article mentions explicitly Neo4j
 the recommended article mentions explicitly Neo4j


title: Performance Testing Neo4j Database using Bolt Protocol in Apache JMeter
explanation: 
the original article mentions explicitly Neo4j
 the recommended article mentions explicitly Neo4j


title: Neo4j: running in Kubernetes
explanation: 
the original article mentions explicitly Neo4j
 the recommended article mentions explicitly Neo4j


title: 9 Best Python Frameworks for Building Small to Enterprise Applications
explanation: 
the original article mentions explicitly Neo4j
 the recommended article mentions explicitly Zope Object Database
Neo4j is a type of graph database
graph database is a type of database management system
Zope Object Database is a type of database management system



In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0)

template = """I've read an article titled {current_article_title}.
Tell me what should I read next and a brief description of why based only on the provided context.

{context}
"""
prompt = PromptTemplate.from_template(template)

theprompt = prompt.format_prompt(current_article_title=current_title,context=context)

#print(theprompt)

llm(theprompt.to_messages())

AIMessage(content='Based on the provided context, the recommended document to read next would be "Building a Modern Web Application with Neo4j and NestJS." This is because the original article explicitly mentions Neo4j, and the recommended article also explicitly mentions Neo4j. This suggests that the recommended article would provide further information and insights related to Neo4j, which would be relevant and beneficial for the reader.')